# Tranform the Github Data  

#### Read the data

In [1]:
# import the libraries
import pandas as pd
import numpy as np
import sys

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# read the data
df = pd.read_csv("../data/total_processed.csv", index_col=0)

/home/geanderson/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,6,7,92) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head(3)

,sha,message,comment_count,date,total_deletions,total_additions,total,comments,additions,deletions,changes,status,filenames,name,fullname,private,fork,size,watchers_count,has_issues,has_downloads,has_wiki,forks_count,open_issues_count,forks,open_issues,watchers,network_count,type,admin,push,pull,owner,language_C,language_C#,language_C++,language_CSS,language_CoffeeScript,language_Go,language_Java,language_JavaScript,language_PHP,language_Perl,language_Python,language_R,language_Ruby,language_Scala,language_Shell,language_TypeScript,label_2.3,label_Awaiting developer action/feedback,label_Bitbucket Open,label_Bug,label_Bug report,label_Build system,label_CodePlex,label_Design,label_Design Decision Needed,label_Feature,label_Feature Request,label_Help,label_New feature,label_RFC,label_Sub-Entities,label_TODO,label_Type - Bug,label_Type-Defect,label_Windows,label_addon,label_bug,label_crash,label_defect,label_doc,label_docs,label_documentation,label_encoding,label_feature,label_feature request,label_feature suggestion,label_feature-request,label_future,label_javascript,label_new icon,label_req,label_reviewed,label_suggestion,label_translator,label_triaged,label_under consideration,label_windows,label_working as intended,total_files,results,dateonly,isweekend,isnight,outside_contribution,tests_included,most_changes_added,most_changes_modified,most_changes_removed,most_changes_renamed,most_changes_unknown
515668,e566ca34a3fd246a4535c458ae47ff7ea62e2de7,New repository initialized by cvs2svn.,0.0,2003-02-13T13:38:33Z,0.0,173,173,[],"[1, 8, 8, 155, 1]","[0, 0, 0, 0, 0]","[1, 8, 8, 155, 1]","[u'added', u'added', u'added', u'added', u'add...","[u'.gitignore', u'pull-binary-libs.sh', u'push...",scala,scala/scala,0.0,0.0,100982.0,1610.0,1.0,1.0,1.0,506.0,15.0,506.0,15.0,1610.0,506.0,0.0,0.0,0.0,0.0,scala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0.0,Thursday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
515665,33d6e170c97ca7b2f991896a0729941a7240b6d6,- Added .cvsignore,0.0,2003-02-13T13:38:33Z,0.0,4,4,[],[4],[0],[4],[u'added'],[u'.cvsignore'],scala,scala/scala,0.0,0.0,100982.0,1610.0,1.0,1.0,1.0,506.0,15.0,506.0,15.0,1610.0,506.0,0.0,0.0,0.0,0.0,scala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,Thursday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
515664,4177daab2f54bdb20c71f623296a8bb32616fd12,Initial version.,0.0,2003-02-13T14:41:36Z,0.0,23138,23138,[],"[87, 221, 354, 38, 45, 169, 96, 123, 30, 430, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[87, 221, 354, 38, 45, 169, 96, 123, 30, 430, ...","[u'added', u'added', u'added', u'added', u'add...","[u'sources/scalac/ApplicationError.java', u'so...",scala,scala/scala,0.0,0.0,100982.0,1610.0,1.0,1.0,1.0,506.0,15.0,506.0,15.0,1610.0,506.0,0.0,0.0,0.0,0.0,scala,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,81,0.0,Thursday,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


### Mean of the commit label

In [5]:
def mean(items):
    index = 0
    result = []
    value = 0

    for item in items:
        index = index + 1

        if item == 1:
            value = value + 1

        result.append((value/index)*100)

    return result

In [6]:
# create a dictionary project_name + mean_values for the commit classification
projects = df['name'].unique()
commit_mean_dict = {}

for project in projects:
    commits = df[df.name==project].results
    mean_commits = mean(commits)
    commit_mean_dict[project] = mean_commits

In [7]:
# replace all the data
df['mean_commits'] = 0

for project in projects:
    df.loc[df['name'] == project, 'mean_commits'] = commit_mean_dict[project]

### Variance of the column results

In [8]:
for project in projects:
    df.loc[df['name'] == project, 'variance_commits'] = df[df['name']==project][['results', 'mean_commits']].var(ddof=0, axis=1)

### Set the phases of the project

In [9]:
def phases(entire_data):
    len_project = len(entire_data)
    phases = []
    
    end_phase_1 = len_project / 3
    end_phase_1 = int(end_phase_1)
    end_phase_2 = end_phase_1 + end_phase_1
    end_phase_2 = int(end_phase_2)

    i = 0

    for item in entire_data:
        if i < end_phase_1:
            phases.append(1)
            i = i + 1
        elif i >= end_phase_1 and i < end_phase_2:
            phases.append(2)
            i = i + 1
        else:
            phases.append(3)
            i = i + 1
        
    return phases

In [10]:
# create a dictionary project_name + phases for the commit classification
commit_phases_dict = {}

for project in projects:
    commits = df[df.name==project].results
    phases_commits = phases(commits)
    commit_phases_dict[project] = phases_commits

In [11]:
# replace all the data
df['phases_project'] = 0

for project in projects:
    df.loc[df['name'] == project, 'phases_project'] = commit_phases_dict[project]

In [12]:
df[df['name']=='stat-cookbook']

,sha,message,comment_count,date,total_deletions,total_additions,total,comments,additions,deletions,changes,status,filenames,name,fullname,private,fork,size,watchers_count,has_issues,has_downloads,has_wiki,forks_count,open_issues_count,forks,open_issues,watchers,network_count,type,admin,push,pull,owner,language_C,language_C#,language_C++,language_CSS,language_CoffeeScript,language_Go,language_Java,language_JavaScript,language_PHP,language_Perl,language_Python,language_R,language_Ruby,language_Scala,language_Shell,language_TypeScript,label_2.3,label_Awaiting developer action/feedback,label_Bitbucket Open,label_Bug,label_Bug report,label_Build system,label_CodePlex,label_Design,label_Design Decision Needed,label_Feature,label_Feature Request,label_Help,label_New feature,label_RFC,label_Sub-Entities,label_TODO,label_Type - Bug,label_Type-Defect,label_Windows,label_addon,label_bug,label_crash,label_defect,label_doc,label_docs,label_documentation,label_encoding,label_feature,label_feature request,label_feature suggestion,label_feature-request,label_future,label_javascript,label_new icon,label_req,label_reviewed,label_suggestion,label_translator,label_triaged,label_under consideration,label_windows,label_working as intended,total_files,results,dateonly,isweekend,isnight,outside_contribution,tests_included,most_changes_added,most_changes_modified,most_changes_removed,most_changes_renamed,most_changes_unknown,mean_commits,variance_commits,phases_project
337,ccc5c3febf1e8e91ef39eee75f931ba3d9cb3714,Initial import.,0.0,2011-03-05T04:48:15Z,0.0,25084,25084,[],"[10, 11, 49, 2519, 258, 851, 738, 4241, 1741, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[10, 11, 49, 2519, 258, 851, 738, 4241, 1741, ...","[u'added', u'added', u'added', u'added', u'add...","[u'.gitignore', u'Makefile', u'cheatsheet.bib'...",stat-cookbook,mavam/stat-cookbook,0.0,0.0,188.0,298.0,1.0,1.0,1.0,41.0,0.0,41.0,0.0,298.0,41.0,1.0,0.0,0.0,0.0,mavam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,0.0,Saturday,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.0
315,ffcd1a8e8f6404d9c294c3d7931874cf70365f9b,Restyle table with booktabs package.\n\nSugges...,0.0,2011-03-05T05:40:40Z,32.0,34,66,[],[34],[32],[66],[u'modified'],[u'cheatsheet.tex'],stat-cookbook,mavam/stat-cookbook,0.0,0.0,188.0,298.0,1.0,1.0,1.0,41.0,0.0,41.0,0.0,298.0,41.0,1.0,0.0,0.0,0.0,mavam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,Saturday,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,1.0
305,7184d955144f7aff75015353b0c999a8463f0430,Add Student's t and F distribution.\n\nSuggest...,0.0,2011-03-05T06:02:25Z,2.0,18,20,[],[18],[2],[20],[u'modified'],[u'cheatsheet.tex'],stat-cookbook,mavam/stat-cookbook,0.0,0.0,188.0,298.0,1.0,1.0,1.0,41.0,0.0,41.0,0.0,298.0,41.0,1.0,0.0,0.0,0.0,mavam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,Saturday,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.000000,1.0
304,2025c1ca462fb3244ab57754954eddcedca0b95a,Move Gamma and Beta function to Math section.,0.0,2011-03-05T06:24:05Z,54.0,47,101,[],[47],[54],[101],[u'modified'],[u'cheatsheet.tex'],stat-cookbook,mavam/stat-cookbook,0.0,0.0,188.0,298.0,1.0,1.0,1.0,41.0,0.0,41.0,0.0,298.0,41.0,1.0,0.0,0.0,0.0,mavam,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0

#### Fill NAN values that can be filled with 0

In [13]:
df['comment_count'] = df['comment_count'].fillna(0)

#### Send the mean_commits to the end

In [14]:
# test the mean_commits as a regression problem
results = df.mean_commits.values
df = df.drop("mean_commits", axis=1)
df['mean_commits'] = results 

In [15]:
# save the csv
df.to_csv('../data/total_transformed.csv')